In [1]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Loading data files

In [2]:
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        
        self.word2index = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        
        self.word2count = {}

        self.n_words = 2
        
    def add_sentence(self, sentence):
        for word in sentence.split(' '):
            self.add_word(word)
            
    def add_word(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.index2word[self.n_words] = word

            self.word2count[word] = 1
            
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [3]:
import unicodedata

def unicode_2_ascii(unicode_string):
    ascii_string = ''
    
    for char in unicodedata.normalize("NFD", unicode_string):
        if unicodedata.category(char) != "Mn":
            ascii_string += char
            
    return ascii_string

In [4]:
import re

def normalize_string(string):
    # Lowercase.
    string = unicode_2_ascii(string.lower())
    # Adds a space btw a letter and a [.!?].
    string = re.sub(r"([.!?])", r" \1", string)
    # Removes non-letter chrs.
    string = re.sub(r"[^a-zA-Z.!?]+", r" ", string)
    
    return string

In [5]:
def read_langs(lang1, lang2, reverse=False):
    print("Reading lines...")
    
    # Reads the file and split into lines.
    with open(f"data/{lang1}-{lang2}.txt", encoding="utf-8") as f:
        lines = f.read().strip().split('\n')
        
    # Splits every line into pairs and normalizes.
    pairs = [[normalize_string(sentence.strip()) for sentence in line.split("\t")]
                for line in lines]
    
    # Reverses pairs, makes Lang instances.
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)
        
    return input_lang, output_lang, pairs

In [6]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

def filter_pair(pair):
    return len(pair[0].split(' ')) < MAX_LENGTH and \
        len(pair[1].split(' ')) < MAX_LENGTH and \
        pair[1].startswith(eng_prefixes)

def filter_pairs(pairs):
    return [pair for pair in pairs if filter_pair(pair)]

In [7]:
import random

def prepare_data(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = read_langs(lang1, lang2, reverse)
    print(f"Read {len(pairs)} sentence pairs")
    
    pairs = filter_pairs(pairs)
    print(f"Trimmed to {len(pairs)} sentence pairs")

    print("Counting words...")
    for pair in pairs:
        input_lang.add_sentence(pair[0])
        output_lang.add_sentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    
    return input_lang, output_lang, pairs

input_lang, output_lang, pairs = prepare_data("eng", "fra", True)
print(random.choice(pairs))

Reading lines...
Read 135842 sentence pairs
Trimmed to 10599 sentence pairs
Counting words...
Counted words:
fra 4345
eng 2803
['je suis heureuse que vous l ayez aime .', 'i m happy you liked it .']


## The Seq2Seq Model

### The Encoder

In [8]:
import torch.nn as nn

class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        
        self.hidden_size = hidden_size
        self.embed_size = self.hidden_size
        
        self.embedding = nn.Embedding(input_size, self.embed_size)
        
        self.gru = nn.GRU(self.embed_size, self.hidden_size)
        
    def forward(self, input_, hidden):
#         output = embedding(input)
#         input: (*), LongTensor of arbitrary shape containing the indices to extract
#         output: (*, H), where * is the input shape and H=\text{embedding\_dim}H=embedding_dim
        embedded = self.embedding(input_).view(1, 1, -1)
        
#         output, h_n = gru(input, h_0)
#         input of shape (seq_len, batch, input_size)
#         h_0 of shape (num_layers * num_directions, batch, hidden_size)
#         output of shape (seq_len, batch, num_directions * hidden_size)
#         h_n of shape (num_layers * num_directions, batch, hidden_size)
        output, hidden = self.gru(embedded, hidden)
        
        return output, hidden
    
    def init_hidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

### The Decoder

#### Simple Decoder

In [9]:
import torch.nn.functional as F

class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        
        self.hidden_size = hidden_size
        self.embed_size = self.hidden_size
        
        # Like a linear?
        self.embedding = nn.Embedding(output_size, self.embed_size)
        
        self.gru = nn.GRU(self.embed_size, self.hidden_size)
        
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, input_, hidden):
        output = self.embedding(input_).view(1, 1, -1)
        output = F.relu(output)
        
        output, hidden = self.gru(output, hidden)
        
        output = self.out(output[0])
        output = self.softmax(output)
        
        return output, hidden
    
    def init_hidden(self):
        return torch.zeros(1, 1, self.hiddden_size, device=device)

#### Attention Decoder

In [10]:
# closer to a lesser-used "location-based" strategy from Luong et al
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        
        self.hidden_size = hidden_size
        self.embed_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

## Training

### Preparing Training Data

In [11]:
def indexes_from_sentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

In [12]:
def tensor_from_sentence(lang, sentence):
    indexes = indexes_from_sentence(lang, sentence)
    indexes.append(EOS_token)  # a list.
    
    # torch.tensor(indexes, dtype=torch.long, device=device) is a 1d tensor

    return torch.tensor(indexes, dtype=torch.long, device=device).unsqueeze_(-1)

In [13]:
def tensors_from_pair(pair):
    input_tensor = tensor_from_sentence(input_lang, pair[0])
    target_tensor = tensor_from_sentence(output_lang, pair[1])
    
    return (input_tensor, target_tensor)

### Training the Model

In [14]:
teacher_forcing_ratio = 0.5

def train(input_tensor, target_tensor, encoder, decoder, criterion, 
              encoder_optimizer, decoder_optimizer, max_length=MAX_LENGTH):
    
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
        
    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)
    
    loss = 0
    
    # Encoder.
    
    # max_length of the sentence is max_length, 
    # each word in the sentence is repr by a vec.
    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    # Encoder hidden.
    encoder_hidden = encoder.init_hidden()
    
    # Clears grads.
    encoder_optimizer.zero_grad()

    for e_idx in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[e_idx], encoder_hidden)

        #         print(encoder_output.size()) is [1, 1, embed_size]
        
        encoder_outputs[e_idx] = encoder_output[0][0]

    # Decoder.
    
    decoder_input = torch.tensor([[SOS_token], device=device)

    # Decoder hidden.
    decoder_hidden = encoder_hidden
    
    # Clears grads.
    decoder_optimizer.zero_grad()
    
    if use_teacher_forcing:
        for d_idx in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            
            loss += criterion(decoder_output, target_tensor[d_idx])
            
            decoder_input = target_tensor[d_idx]  # Teacher forcing

    else:
        for d_idx in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            
#             decoder_output.size() is [1, lang2.n_words]
#             decoder_output.topk(k) -> [1, k], [1, k]
            
            top_values, top_indexes = decoder_output.topk(1)
            
#             top_indexes.size() is [1, 1]
#             top_indexes.squeeze() removes '1' in top_indexes.size()
            
            decoder_input = top_indexes.squeeze().detach()  # detach from history as input
        
            loss += criterion(decoder_output, target_tensor[d_idx])
            
            if decoder_input.item() == EOS_token:
                break

    # Backward.
    loss.backward()

    # Upgrads params.
    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [15]:
import time

def as_minutes(total_secs):
    minutes = int(total_secs / 60)
    seconds = total_secs - minutes * 60
    
    return f"{minutes:.0f}m {seconds:.0f}s"

In [16]:
def time_since(since, percent):
    now = time.time()
    
    total_secs = now - since
    entire_secs = total_secs / percent
    remaining_secs = entire_secs - total_secs
    
    return f"{as_minutes(total_secs)} (- {as_minutes(remaining_secs)})"

In [17]:
import torch.optim as optim

def train_iters(encoder, decoder, n_iters, learning_rate=0.01, print_every=1000, plot_every=100):
    # Init of vars for tracking the training process.
    start = time.time()
    
    print_loss_total = 0
    plot_losses = []
    plot_loss_total = 0

    # Loss func.
    criterion = nn.NLLLoss()
    
    # Optimizers.
    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)

    training_pairs = [tensors_from_pair(random.choice(pairs))
                      for i in range(n_iters)]

    for iter in range(1, n_iters + 1):
        # Gets a training example.
        input_tensor, target_tensor = training_pairs[iter - 1]

        #         print(input_tensor)
        #         print(target_tensor)
#         tensor([[351],
#         [349],
#         [376],
#         [  5],
#         [  1]], device='cuda:0')
#         tensor([[221],
#         [ 78],
#         [ 72],
#         [  4],
#         [  1]], device='cuda:0')
        
        # Training.
        loss = train(input_tensor, target_tensor, encoder,
                     decoder, criterion, encoder_optimizer, decoder_optimizer)

        # Tracks the training process.
        print_loss_total += loss
        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            percent = iter / n_iters
            print(f"{time_since(start, percent)} ({iter} {percent:.0%}) {print_loss_avg:.4f}")
            print_loss_total = 0
        
        plot_loss_total += loss
        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    show_plot(plot_losses)

### Plotting results

In [18]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

def show_plot(losses):
    plt.figure()
    fig, ax = plt.subplots()
    
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    
    plt.plot(losses)
    plt.show()

## Evaluation

In [19]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        
        # Encoder. 
        
        input_tensor = tensor_from_sentence(input_lang, sentence)
        
        input_length = input_tensor.size()[0]
        
        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)
                
        encoder_hidden = encoder.init_hidden()

        for e_idx in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[e_idx],
                                                     encoder_hidden)
            
            encoder_outputs[e_idx] += encoder_output[0, 0]

        # Decoder.
        
        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)
        
        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        for d_idx in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            
            decoder_attentions[d_idx] = decoder_attention.data
            
            top_values, top_idxs = decoder_output.data.topk(1)
            
            decoder_input = top_idxs.squeeze().detach()
            
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[top_idxs.item()])

        return decoded_words, decoder_attentions[:d_idx + 1]

In [20]:
def evaluate_randomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

## Training and Evaluating

In [21]:
hidden_size = 256

encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

In [ ]:
# Training.
train_iters(encoder, attn_decoder, n_iters=75_000, print_every=5000)

2m 16s (- 31m 50s) (5000 7%) 2.8381
4m 29s (- 29m 11s) (10000 13%) 2.2792
6m 44s (- 26m 57s) (15000 20%) 1.9879
8m 59s (- 24m 42s) (20000 27%) 1.7277
11m 16s (- 22m 31s) (25000 33%) 1.5311


In [ ]:
# Evaluating.
evaluate_randomly(encoder, attn_decoder)

### Visualizing Attention

In [ ]:
def show_attention(input_sentence, output_words, attentions):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    
    # Shows the colored matrix.
    color_ax = ax.matshow(attentions)
    
    # Shows a color bar.
    fig.colorbar(color_ax)
    
    # Sets up axes.
    ax.set_xticklabels([''] + input_sentence.split(' ') + ['<EOS>'], rotation=90)
    ax.set_yticklabels(['' + output_words])
    
    # Shows labels at every tick.
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))
    
    plt.show()

In [ ]:
def evaluate_and_show_attention(input_sentence):
    output_words, attentions = evaluate(encoder, attn_decoder, input_sentence)
    
    print("input =", input_sentence)
    print("output =", ' '.join(output_words))
    
    show_attention(input_sentence, output_words, attentions)

In [ ]:
evaluateAndShowAttention("elle a cinq ans de moins que moi .")

evaluateAndShowAttention("elle est trop petit .")

evaluateAndShowAttention("je ne crains pas de mourir .")

evaluateAndShowAttention("c est un jeune directeur plein de talent .")